In [2]:
SEED = 0

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"
%cd /content/gdrive/My Drive/Kaggle


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Kaggle


In [ ]:
!kaggle datasets download -d mlg-ulb/creditcardfraud


In [ ]:
!unzip \*.zip  && rm *.zip
!ls

In [5]:
#Loading data
import pandas as pd
import numpy as np

data = pd.read_csv("/content/gdrive/My Drive/Kaggle/creditcard.csv", header=0)
data.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/gdrive/My Drive/Kaggle/creditcard.csv'

In [4]:
#Handle Duplicated values
print("Duplicates", data.duplicated().sum())
data = data.drop_duplicates()
print("Rows {}, Columns {}".format(data.shape[0], data.shape[1]))

NameError: name 'data' is not defined

In [ ]:
data['Class'] = data['Class'].replace({'0': 0, '1': 1})
data['Class'].unique()
data['Class'].value_counts()

0    283253
1       473
Name: Class, dtype: int64

In [ ]:
labels = data['Class'] 
data = data[data.columns[:-1]]
labels.value_counts()

0    283253
1       473
Name: Class, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( data, labels, test_size=0.3, random_state=SEED, stratify = labels )

In [ ]:
#Bagging using Random forest

from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.ensemble import BalancedBaggingClassifier

# Define the base classifier
base_clf = RandomForestClassifier()

# Define the bagging classifier
bagging_clf = BaggingClassifier(base_estimator=base_clf, n_estimators=10)

# Train the bagging classifier on the training data
bagging_clf.fit(X_train, y_train)

# Evaluate the bagging classifier on the test data
y_pred = bagging_clf.predict(X_test)
print(classification_report(y_test, y_pred))


/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     84976
           1       0.98      0.80      0.88       142

    accuracy                           1.00     85118
   macro avg       0.99      0.90      0.94     85118
weighted avg       1.00      1.00      1.00     85118



In [ ]:
#Boosting using Decision Tree
from sklearn.ensemble import AdaBoostClassifier, DecisionTreeClassifier

from sklearn.metrics import classification_report

# Define the base classifier
base_clf = DecisionTreeClassifier()

# Define the boosting classifier
boosting_clf = AdaBoostClassifier(base_estimator=base_clf, n_estimators=50, learning_rate=1)

# Train the boosting classifier on the training data
boosting_clf.fit(X_train, y_train)

# Evaluate the boosting classifier on the test data
y_pred = boosting_clf.predict(X_test)
print(classification_report(y_test, y_pred))


/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     84976
           1       0.72      0.77      0.74       142

    accuracy                           1.00     85118
   macro avg       0.86      0.88      0.87     85118
weighted avg       1.00      1.00      1.00     85118

Confusion Matrix:
 [[84934    42]
 [   33   109]]


In [1]:
#Ada boost with weight balanced
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Define AdaBoost classifier and specify the minority class weight
ada = AdaBoostClassifier(n_estimators=50, random_state=SEED, algorithm='SAMME.R', 
                         learning_rate=1.0, class_weight='balanced')

# Train the classifier
ada.fit(X_train, y_train)

# Predict on test data
y_pred = ada.predict(X_test)

# Evaluate performance
from sklearn.metrics import confusion_matrix, classification_report
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


NameError: name 'SEED' is not defined

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
from imblearn.over_sampling import RandomOverSampler

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)  # Compute class ratio
xgb_clf = XGBClassifier(scale_pos_weight=ratio, random_state=SEED)

# Train the classifier
xgb_clf.fit(X_train, y_train)

# Predict on test data
y_pred = xgb_clf.predict(X_test)

# Evaluate performance
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Confusion Matrix:
 [[84970     6]
 [   23   119]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     84976
           1       0.95      0.84      0.89       142

    accuracy                           1.00     85118
   macro avg       0.98      0.92      0.95     85118
weighted avg       1.00      1.00      1.00     85118



In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
print("\n", ConfusionMatrixDisplay.from_estimator(xgb_clf, X_test_pca, y_test, display_labels=labels.unique()))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-41-3552819214f8>", line 2, in <cell line: 2>
    print("\n", ConfusionMatrixDisplay.from_estimator(xgb_clf, X_test_pca, y_test, display_labels=labels.unique()))
NameError: name 'X_test_pca' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_l